In [ ]:
## Análise de distribuição estatística dos dados - Histograma, normalidade e transformação Box-Cox

## CÉLULA 1: Instruções de carregamento do arquivo e bibliotecas a ser importadas

""""
    Para executar uma CÉLULA, pressione o botão "Run" no topo, com a CÉLULA selecionada, ou pressione:
    
    Shift + Enter

-Se seus dados estiverem em um arquivo xlsx utilize a CÉLULA 2

-Se seus dados estiverem em um arquivo csv contendo cabeçalhos, utilize a CÉLULA 3

-Se seus dados estiverem em um arquivo csv sem cabeçalhos, utilize a CÉLULA 4

-Utilize apenas a CÉLULA correta
-Substitua o texto entre aspas na variável "caminho" pelo endereço do arquivo em sua máquina.

EXECUTE ESTA CÉLULA ANTES DE PROSSEGUIR

EXECUTE APENAS UMA CÉLULA POR VEZ

-Caso deseje apagar a saída de uma célula carregada, mas não deseje carregar uma nova saída, vá à aba superior, escolha o ícone
do teclado (open the command palette/ jupyter-notebook command group) e selecione clear cell output.

@author: Marco César Prado Soares, MSc.
Especialista Lean Six Sigma Master Black Belt, Eng. Químico, MSc. Eng. Mecatrônica (instrumentação) 
Marco.Soares@br.ey.com; marcosoares.feq@gmail.com
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## CÉLULA 2: dados em arquivo xlsx

caminho = "D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 2 - ANN regression\ANN.1 - Apartment price analysis\prices_apartments.xlsx"
dataset = pd.read_excel(caminho)

dataset
#SIGA PARA A CÉLULA 5

In [ ]:
## CÉLULA 3: dados em arquivo csv com cabeçalho

caminho = "D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 3 - ANN classification\ANN.Class - 2 - advertising campaign\delivery_app_data.csv"
dataset = pd.read_csv(caminho)

dataset
#SIGA PARA A CÉLULA 5

In [ ]:
## CÉLULA 4: dados em arquivo csv sem cabeçalho

caminho = "D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 2 - ANN regression\ANN.4 - Boston housing price\housing_boston_data.txt"
dataset = pd.read_csv(caminho, delim_whitespace=True, header=None)

#here, the dataset is in a textfile. In these cases, use delim_withespace = True
#or df = pd.read_fwf('output_list.txt')
#since data has no Head, keep header = None
#if there is Head, eliminate ‘header’ from the arguments of the function

dataset
#SIGA PARA A CÉLULA 5

In [ ]:
## CÉLULA 5: Eliminar linhas com entradas nulas

"""
-Utilize este CÉLULA apenas se desejar limpar as entradas nulas da sua base de dados

-Esta CÉLULA eliminará todas as linhas contendo entradas nulas. Caso NÃO DESEJE ISTO, vá direto à CÉLULA 6, sem executar esta
célula.
"""

dataset = dataset.dropna(axis=0)
dataset
#SIGA PARA A CÉLULA 6

# CÉLULA 6: Instruções Gerais

### ATENÇÃO: NÃO EXECUTE ESTA CÉLULA - Ela contém apenas texto explicativo

Este notebook destina-se à análise de distribuição estatística dos dados, e transformação deles em dados que sigam a distribuição normal. À seguir, são dadas as análises disponíveis. Siga à célula correspondente à análise desejada:

## 1) Histograma com ou sem curva normal ajustada

- Aqui será construído o histograma dos dados.
- Este sistema permite a verificação simultânea da curva normal ajustada.
- O resultado pode ser prejudicado para dados com diferenças extremamente pequenas. Neste caso, pode ser mais interessante utilizar o comando de histograma do R (exemplo: hist(y,main="Histograma do valor de inventario", xlab = "Valor", ylab = "Contagem", col="red"), o qual já é otimizado para esta situação particular.
- Também são retornados: 
    1) um resumo das estatísticas gerais dos dados: total de dados avaliados; média; desvio-padrão; valor mais elevado; valor 
        mais baixo; e bin size, a largura da barra do histograma.
    2) a tabela de frequencias usada na construção do histograma

Siga para as CÉLULAS 7 a 9.
----------------------------------------

_______________________________________________________________________________________________________________________________
_______________________________________________________________________________________________________________________________
_______________________________________________________________________________________________________________________________

## 2) Testes de normalidade

- Aqui será avaliado se os dados podem ser descritos por uma distribuição estatística do tipo normal, por meio de dois testes distintos: teste de Lilliefors e teste de Anderson-Darling.
- Note que existem outros testes (em especial testes Anderson-Darling) destinados à avaliação de outras distribuições. O pacote nortest do R fornece uma gama de testes deste tipo, em variedade maior que o disponibilizado no pacote statsmodels do Python.
- Este sistema retornará dois dataframes contendo p-valores, a probabilidade de uma hipótese-nula ser verdadeira. Neste caso, a hipótese-nula dos testes de hipótese (Lilliefors e Anderson-Darling) é que os dados seguem a normal.

Siga para as CÉLULAS 10 a 12.
----------------------------------------

_______________________________________________________________________________________________________________________________
_______________________________________________________________________________________________________________________________
_______________________________________________________________________________________________________________________________

## 3) Transformação Box-Cox

- Este sistema transforma o conjunto de dados original em dados descritos pela curva normal.
- Também os limites de especificação transformados.
- Possibilita, assim, que seja analisado se os dados estão dentro das especificações.

Siga para as CÉLULAS 13 a 15.
----------------------------------------

In [ ]:
## CÉLULA 7: Função de construção do histograma dos dados

#Execute esta CÉLULA para carregar a função

def histogram(y, largura_da_barra, normal_curve_overlay = True, legenda_dos_dados = None, titulo_y = None, titulo_histograma = None):
    
    import pandas as pd
    import matplotlib
    import numpy as np
    import matplotlib.pyplot as plt
    
    #Calculo do bin size - largura do histograma:
    #1: Encontrar o menor (lowest) e o maior (highest) valor dentro da tabela de dados)
    #2: Calcular rangehist = highest - lowest
    #3: Calcular quantidade de dados (samplesize) de entrada fornecidos
    #4: Calcular a quantidade de celulas da tabela de frequencias (ncells)
    #ncells = numero inteiro mais proximo da (raiz quadrada de samplesize)
    #5: Calcular binsize = rangehist/(ncells)
    #ATENCAO: Nao se esquecer de converter range, ncells, samplesize e binsize para valores absolutos (modulos)
    #isso porque a largura do histograma tem que ser um numero positivo

    y = y.reset_index(drop=True)
    #faz com que os indices desta serie sejam consecutivos e a partir de zero

    #Estatisticas gerais: media (mu) e desvio-padrao (sigma)
    mu = y.mean() 
    sigma = y.std() 

    #Calculo do bin-size
    highest = y.max()
    lowest = y.min()
    rangehist = highest - lowest
    rangehist = abs(rangehist)
    #garante que sera um numero positivo
    samplesize = y.count() #contagem do total de entradas
    ncells = (samplesize)**0.5 #potenciacao: ** - raiz quadrada de samplesize
    #resultado da raiz quadrada e sempre positivo
    ncells = round(ncells) #numero "redondo" mais proximo
    ncells = int(ncells) #parte inteira do numero arredondado
    #ncells = numero de linhas da tabela de frequencias
    binsize = rangehist/ncells
    binsize = round(binsize)
    binsize = int(binsize) #precisa ser inteiro
    
    #Construcao da tabela de frequencias

    j = 0 #indice da tabela de frequencias
    #Este indice e diferente do ordenamento dos valores em ordem crescente
    xhist = []
    #Lista vazia que contera os x do histograma
    yhist = []
    #Listas vazia que conteras o y do histograma
    hist_labels = []
    #Esta lista gravara os limites da barra na forma de strings

    pontomediodabarra = lowest + binsize/2 
    limitedabarra = lowest + binsize
    #ponto medio da barra 
    #limite da primeira barra do histograma
    seriedohist1 = y
    seriedohist1 = seriedohist1.sort_values(ascending=True)
    #serie com os valores em ordem crescente
    seriedohist1 = seriedohist1.reset_index(drop=True)
    #garante que a nova serie tenha indices consecutivos, iniciando em zero
    i = 0 #linha inicial da serie do histograma em ordem crescente
    valcomparado = seriedohist1[i]
    #primeiro valor da serie, o mais baixo

    while (j <= (ncells-1)):
        
        #para quando termina o numero de linhas da tabela
        xhist.append(pontomediodabarra)
        #tempo da tabela de frequencias
        cont = 0
        #variavel de contagem do histograma
        #contagem deve ser reiniciada
       
        if (i < samplesize):
            #2 condicionais para impedir que um termo de indice inexistente
            #seja acessado
            while (valcomparado <= limitedabarra) and (valcomparado < highest):
                #o segundo criterio garante a parada em casos em que os dados sao
                #muito proximos
                    cont = cont + 1 #adiciona contagem a tabela de frequencias
                    i = i + 1
                    
                    if (i < samplesize): 
                        valcomparado = seriedohist1[i]
        
        yhist.append(cont) #valor de ocorrencias contadas
        
        limite_infdabarra = pontomediodabarra - binsize/2
        rotulo = "%.2f - %.2f" %(limite_infdabarra, limitedabarra)
        #intervalo da tabela de frequencias
        #%.2f: 2 casas decimais de aproximação
        hist_labels.append(rotulo)
        
        pontomediodabarra = pontomediodabarra + binsize
        #tanto os pontos medios quanto os limites se deslocam do mesmo intervalo
        
        limitedabarra = limitedabarra + binsize
        #proxima barra
        
        j = j + 1
    
    #Temos que verificar se o valor maximo foi incluido
    #isso porque o processo de aproximacao por numero inteiro pode ter
    #arredondado para baixo e excluido o limite superior
    #Porem, note que na ultima iteracao o limite superior da barra foi 
    #somado de binsize, mas como j ja e maior que ncells-1, o loop parou
    
    #assim, o limitedabarra nesse momento e o limite da barra que seria
    #construida em seguida, nao da ultima barra da tabela de frequencias
    #isso pode fazer com que esta barra ja seja maior que o highest
    
    #note porem que nao aumentamos o valor do limite inferior da barra
    #por isso, basta vermos se ele mais o binsize sao menores que o valor mais alto
    
    
    while ((limite_infdabarra+binsize) < highest):
        
        #vamos criar novas linhas ate que o ponto mais alto do histograma
        #tenha sido contado
        ncells = ncells + 1 #adiciona uma linha a tabela de frequencias
        xhist.append(pontomediodabarra)
        
        cont = 0 #variavel de contagem do histograma
        
        while (valcomparado <= limitedabarra):
                cont = cont + 1 #adiciona contagem a tabela de frequencias
                i = i + 1
                if (i < samplesize):
                    valcomparado = seriedohist1[i]
                    #apenas se i ainda nao e maior que o total de dados
                
                else: 
                    
                    break
        
        #parar o loop se i atingiu um tamanho maior que a quantidade 
        #de dados.Temos que ter este cuidado porque estamos acrescentando
        #mais linhas a tabela de frequencias para corrigir a aproximacao
        #de ncells por um numero inteiro
        
        yhist.append(cont) #valor de ocorrencias contadas
        
        limite_infdabarra = pontomediodabarra - binsize/2
        rotulo = "%.2f - %.2f" %(limite_infdabarra, limitedabarra)
        #intervalo da tabela de frequencias - 2 casas decimais
        hist_labels.append(rotulo)
        
        pontomediodabarra = pontomediodabarra + binsize
        #tanto os pontos medios quanto os limites se deslocam do mesmo intervalo
        
        limitedabarra = limitedabarra + binsize
        #proxima barra
        
    estatisticas_col1 = []
    #contera as descricoes das colunas da tabela de estatisticas gerais
    estatisticas_col2 = []
    #contera os valores da tabela de estatisticas gerais
    
    estatisticas_col1.append("Total de dados avaliados")
    estatisticas_col2.append(samplesize)
    estatisticas_col1.append("Média (mu)")
    estatisticas_col2.append(mu)
    estatisticas_col1.append("Desvio-padrão (sigma)")
    estatisticas_col2.append(sigma)
    estatisticas_col1.append("Valor mais elevado")
    estatisticas_col2.append(highest)
    estatisticas_col1.append("Valor mais baixo")
    estatisticas_col2.append(lowest)
    estatisticas_col1.append("Range dos dados\n(valor máximo - valor mínimo)")
    estatisticas_col2.append(rangehist)
    estatisticas_col1.append("Bin size\n(largura da barra do histograma)")
    estatisticas_col2.append(binsize)
    estatisticas_col1.append("Quantidade de linhas\nna tabela de frequências")
    estatisticas_col2.append(ncells)
    #como o comando append grava linha a linha em sequencia, garantimos
    #a correspondencia das colunas
    #Assim como em qualquer string, incluindo de rotulos de graficos
    #os \n sao lidos como quebra de linha
    
    d1 = {"Estatísticas gerais dos dados": estatisticas_col1, "Valor calculado": estatisticas_col2}
    #dicionario das duas series, para criar o dataframe com as descricoes
    estatisticas_gerais = pd.DataFrame(data = d1)
    
    #Casos os títulos estejam presentes (valor nao e None):
    #vamos utiliza-los
    #Caso contrario, vamos criar nomenclaturas genericas para o histograma
    
    eixo_y = "Counting/Frequency"
    
    if not (legenda_dos_dados is None):
        xlabel = legenda_dos_dados
    
    else:
        xlabel = "Frequency\n table data"
    
    if not (titulo_y is None):
        eixo_x = titulo_y
        #lembre-se que no histograma, os dados originais vao pro eixo X
        #O eixo Y vira o eixo da contagem/frequencia daqueles dados
    
    else:
        eixo_x = "X: Mean value of the interval"
    
    if not (titulo_histograma is None):
        string1 = "- $\mu = %.2f$, $\sigma = %.2f$" %(mu, sigma)
        main_label = titulo_histograma + string1
        #concatena a string do titulo a string com a media e desvio-padrao
        #%.2f: o numero entre %. e f indica a quantidade de casas decimais da 
        #variavel float f. No caso, arredondamos para 2 casas
        #NAO SE ESQUECA DO PONTO: ele que indicara que sera arredondado o 
        #numero de casas
    
    else:
        main_label = "Data Histogram - $\mu = %.2f$, $\sigma = %.2f$" %(mu, sigma)
        #os simbolos $\ $ substituem o simbolo pela letra grega
    
    d2 = {"Intervalos considerados": hist_labels, eixo_x: xhist, eixo_y: yhist}
    #dicionario que compoe a tabela de frequencias
    tab_frequencias = pd.DataFrame(data = d2)
    #cria a tabela de frequencias como um dataframe de saida
    
    """
    Normal curve equation
    
    y = p(X) = (1/((sigma)*sqrt(2*pi)))*exp((-1)*(X-mu)²/2(sigma²))
    0 <= p(X) <= 1
    mu = mean value of X
    sigma = standard deviation of X
    sqrt = square root function
    pi = 3.14159...
    e = 2.71828...
    exp = e** = e^ = exponential function
    
    numpy functions available:
        np.sqrt(X): square root of X
        np.pi = value of pi
        np.exp(X) = exponential function of X = e**X = e^X = exp(X)
    
    """
    #parametros da normal ja calculados:
    #mu e sigma
    #numero de bins: ncells
    #limites de especificacao: lsl,usl - target
    
    #valor maximo do histograma
    max_hist = max(yhist)
    #seleciona o valor maximo da serie, para ajustar a curva normal
    #isso porque a normal é criada com valores entre 0 e 1
    #multiplicando ela por max_hist, fazemos ela se adequar a altura do histograma
    
    if (normal_curve_overlay == True):
        
        #construir a normal ajustada/esperada
        #vamos criar pontos ao redor da media mu - 4sigma ate mu + 4sigma, 
        #de modo a garantir a quase totalidade da curva normal. 
        #O incremento será de 0.10 sigma a cada iteracao
        x_inf = mu -(4)*sigma
        x_sup = mu + 4*sigma
        x_inc = (0.10)*sigma
        
        x_normal_adj = []
        y_normal_adj = []
        
        x_adj = x_inf
        y_adj = ((1 / (np.sqrt(2 * np.pi) * sigma)) *np.exp(-0.5 * (1 / sigma * (x_adj - mu))**2))
        x_normal_adj.append(x_adj)
        y_normal_adj.append(y_adj)
        
        while(x_adj < x_sup): 
            
            x_adj = x_adj + x_inc
            y_adj = ((1 / (np.sqrt(2 * np.pi) * sigma)) *np.exp(-0.5 * (1 / sigma * (x_adj - mu))**2))
            x_normal_adj.append(x_adj)
            y_normal_adj.append(y_adj)
        
        #vamos ajustar a altura da curva ao histograma. Para isso, precisamos
        #calcular quantas vezes o ponto mais alto do histograma é maior que o ponto
        #mais alto da normal (chamaremos essa relação de fator). A seguir,
        #multiplicamos cada elemento da normal por este mesmo fator
        max_normal = max(y_normal_adj) 
        #maximo da normal ajustada, numero entre 0 e 1
        
        fator = (max_hist)/(max_normal)
        size_normal = len(y_normal_adj) #quantidade de dados criados
        
        i = 0
        while (i < size_normal):
            y_normal_adj[i] = (y_normal_adj[i])*(fator)
            i = i + 1
    
    #Fazer o grafico
    fig, ax = plt.subplots()
    
    ax.bar(xhist, yhist, width = largura_da_barra, label=xlabel, color='blue')
    #ajuste manualmente a largura, width, para deixar as barras mais ou menos proximas
    
    if (normal_curve_overlay == True):
    
        #adicionar a normal
        ax.plot(x_normal_adj, y_normal_adj, color = 'black', label = 'Adjusted/expected\n normal curve')
    
    ax.set_xlabel(eixo_x)
    ax.set_ylabel(eixo_y)
    ax.set_title(main_label)
    ax.set_xticks(xhist)
    
    ax.legend()
    ax.grid(True) #mude para False, caso não deseje ver as linhas de grade
    
    fig.tight_layout()
    plt.show()

    return estatisticas_gerais, tab_frequencias

# CÉLULA 8: Função para construção do histograma e da tabela de frequência dos dados

### ATENÇÃO: NÃO EXECUTE ESTA CÉLULA - Ela contém apenas texto explicativo    
    
INSTRUÇÕES:

1) Esta função retorna 2 dataframes:
    
    1) dataframe estatisticas_gerais, um resumo das estatísticas gerais dos dados: total de dados avaliados; média; desvio-padrão; valor mais elevado; valor mais baixo; e bin size, a largura da barra do histograma.
    
    2) dataframe tab_frequencias, que mostra a tabela de frequencias usada na construção do histograma

2) Os parâmetros x e y da função devem ser apenas listas, não dataframe(s)

3) Esta função retorna 2 dataframes. Deste modo, você precisa chamar 2 daframes, não apenas um.

Exemplo: caso deseje salvar o dataframe estatisticas_gerais em df1,  e o dataframe tab_frequencias em df2, e os dados estão na variável y:
    
       df1, df2 = histogram(y, largura_da_barra = 10, normal_curve_overlay = True, legenda_dos_dados = None, titulo_y = None, titulo_histograma = None)
    
Note que o primeiro será sempre estatisticas_gerais, e o segundo será tab_frequencias

Você pode dar outros nomes para os dataframes chamados:
   
       estatisticas_gerais, tab_frequencias = histogram(y, largura_da_barra = 10, normal_curve_overlay = True, legenda_dos_dados = None, titulo_y = None, titulo_histograma = None)

4) Caso QUEIRA mostrar a curva normal ajustada, mantenha:
    
        normal_curve_overlay = True

CASO NAO QUEIRA MOSTRAR A NORMAL:
    
        normal_curve_overlay = False

5) Altere o valor de largura_da_barra até conseguir ver o histograma de forma clara
    
6) Os demais parâmetros são textos (strings). Declare-os entre aspas ou mantenha o valor None
    
##### NOTA: ESTE ALGORITMO PODE FALHAR CASO TODOS OS DADOS SEJAM MUITO PRÓXIMOS. 

Isso porque a largura ncells teria de ser tão larga que uma única barra englobaria todos os dados. Além disso, principalmente quando se usa a transformação Box-Cox, a diferença entre os dados transformados pode ser baixa demais para que a memória 
disponível no sistema identifique eles como números efetivamente diferentes (no arredondamento, eles passam a ser considerados iguais, fazendo com que o loop de comparação que verifica se um dado está numa barra ou na seguinte não possa ser finalizado. Isto gera MemoryError).

CASO QUEIRA, mesmo assim, CONSTRUIR O HISTOGRAMA, apague as linhas de cálculo do bin size (de binsize = rangehist/ncells até binsize = int(binsize)) e substitua por um valor desejado de binsize. Por exemplo, substitua as linhas por:
    
    binsize = 10

Assim, você definirá manualmente o tamanho da barra, podendo "esticá-la" para englobar todos os dados.

In [ ]:
## CÉLULA 9: CONSTRUÇÃO DO HISTOGRAMA PARA SEUS DADOS

#DEFINA A VARIAVEL y para a qual será construído o histograma. 
#Basta substituir o valor entre aspas pelo nome da coluna onde está Y:

y = dataset['Y']

#NOTA: o valor y necessariamente deve ser uma série.

estatisticas_gerais, tab_frequencias = histogram(y = y, largura_da_barra = 10, normal_curve_overlay = True, legenda_dos_dados = None, titulo_y = None, titulo_histograma = None)

# Altere manualmente o valor numérico da largura_da_barra do histograma até alcançar um espaçamento mínimo entre barras
#consecutivas. O valor da largura da barra depende de cada conjunto particular de dados utilizado.

"""
Substitua os demais campos None de acordo com as instruções dadas na CÉLULA 8.
- Os títulos e legendas devem ser fornecidos como um texto entre aspas.
- Modifique normal_curve_overlay = True para normal_curve_overlay = False caso não deseje ver a normal sobreposta.

"""

estatisticas_gerais
tab_frequencias

"""VOCÊ DESEJA EXPORTAR OS DADOS?
Caso deseje exportar os dados, copie as seguintes linhas para o espaço não-vermelho após as aspas. Substitua o endereço pela
pasta onde você deseja salvar seu arquivo. Substitua estatisticas e tabela_freq pelos nomes que deseja para seus arquivos. 
Mantenha a extensão csv. Você pode também optar por exportar apenas uma das tabelas (neste caso, copie apenas a desejada).

estatisticas_gerais.to_csv(r"D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 3 - ANN classification\ANN.Class - 1 - user satisfaction\estatisticas.csv", index = False)

tab_frequencias.to_csv(r"D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 3 - ANN classification\ANN.Class - 1 - user satisfaction\tabela_freq.csv", index = False)

"""

In [ ]:
## CÉLULA 10: Função de avaliação da normalidade dos dados(probabilidade de os dados serem representados por uma curva normal)

#Execute esta CÉLULA para carregar a função

def testes_normalidade(y, alpha = 0.10):
    import pandas as pd
    from statsmodels.stats import diagnostic
    
    lista1 = []
    #esta lista sera a primeira coluna, com as descrições das demais
    lista1.append("p-valor: probabilidade de os dados serem descritos pela normal")
    lista1.append("Probabilidade de seguir a normal (%)")
    lista1.append("alpha")
    lista1.append("Critério: não segue a normal se p < alpha = %.3f" %(alpha))
    #%.3f apresenta f com 3 casas decimais
    #%f se refere a uma variavel float
    #informa ao usuario o valor definido para a rejeição
    lista1.append("Dados são descritos ou não pela normal?")
    #Note que o comando append adiciona os elementos em sequencia, linha a linha
    #nao se especifica indice, pois ja esta subentendido que esta na proxima
    #linha
    
    #Lilliefors’ test
    lilliefors_test = diagnostic.kstest_normal(y, dist='norm', pvalmethod='table')
    #Return: linha 1: ksstat: float
    #Kolmogorov-Smirnov test statistic with estimated mean and variance.
    #Linha 2: p-value:float
    #If the pvalue is lower than some threshold, e.g. 0.10, then we can reject the Null hypothesis that the sample comes from a normal distribution.
    
    #criar lista apenas com o p-valor
    p_lillie = []
    p_lillie.append(lilliefors_test[1]) #p-valor
    p_lillie.append(100*lilliefors_test[1]) #p em porcentagem
    p_lillie.append(alpha)
    
    if (lilliefors_test[1] < alpha):
        p_lillie.append("p = %.3f < %.3f" %(lilliefors_test[1], alpha))
        p_lillie.append("Dados não são descritos pela normal")
    else:
        p_lillie.append("p = %.3f >= %.3f" %(lilliefors_test[1], alpha))
        p_lillie.append("Dados são descritos pela normal")
        
    
    #Anderson-Darling
    ad_test = diagnostic.normal_ad(y, axis=0)
    #Return: Linha 1: ad2: float
    #Anderson Darling test statistic.
    #Linha 2: p-val: float
    #The p-value for hypothesis that the data comes from a normal distribution with unknown mean and variance.
    
    #criar lista apenas com o p-valor
    p_ad = []
    p_ad.append(ad_test[1]) #p-valor
    p_ad.append(100*ad_test[1]) #p em porcentagem
    p_ad.append(alpha)
    
    if (ad_test[1] < alpha):
        p_ad.append("p = %.3f < %.3f" %(ad_test[1], alpha))
        p_ad.append("Dados não são descritos pela normal")
    else:
        p_ad.append("p = %.3f >= %.3f" %(ad_test[1], alpha))
        p_ad.append("Dados são descritos pela normal")
    
    #NOTA: o comando %f apresenta a variavel float com todas as casas
    #decimais possiveis. Se desejamos um numero certo de casas decimais
    #acrescentamos esse numero a frente. Exemplos: %.1f: 1 casa decimal
    # %.2f: 2 casas; %.3f: 3 casas decimais, %.4f: 4 casas
    
    d = {'Parâmetros e Interpretações': lista1, 'Teste de Lilliefors': p_lillie, 'Teste de Anderson-Darling': p_ad}
    
    #dicionario dos valores obtidos
    df = pd.DataFrame(data = d)
    #dataframe de saída
    
    return df

# CÉLULA 11: Função para avaliar normalidade dos dados

### ATENÇÃO: NÃO EXECUTE ESTA CÉLULA - Ela contém apenas texto explicativo

Avaliação se os dados seguem a distribuição normal, de acordo com os testes de Lilliefors e Anderson-Darling

INSTRUÇÕES:

1) Esta função retorna 1 dataframe:
    dataframe df1 contendo os p-valores dos testes de normalidade dos dados.

2) O parâmetro y da função deve ser apenas uma lista, não um dataframe.

3) p-valor: probabilidade da hipótese-nula do teste de hipótese ser verdadeira

Hipótese-nula H0: os dados seguem uma distribuição normal
Hipótese alternativa H1: os dados não seguem a normal

4) o p-valor é uma probabilidade, i.e., um valor entre 0 e 1

5) 100 vezes o p-valor é a probabilidade em porcentagem
        
        Exemplo: se p = 0.20, a probabilidade é 0.20 ou 20% de os dados serem descritos pela normal

6) Critério de rejeição: define-se um valor alpha tal que, se p < alpha, rejeitamos a hipótese-nula, ou seja, consideramos que os dados não seguem a normal

###### 7) em geral, tomamos alpha = 0.10 - ou seja, se p < 0,10 (se há menos de 10% de probabilidade de seguirem a normal), consideramos que os dados não seguem a distribuição

##### 8) Alguns autores são mais rigorosos, considerando alpha = 0.05 - ou seja, só rejeitam a normal se existe menos de 5% de probabilidade de os dados serem descritos por uma normal.

### PORTANTO: MANTENHA alpha = 0.10 
                
                OU 

### ALTERE para alpha = 0.05

In [ ]:
## CÉLULA 12: AVALIAÇÃO DE NORMALIDADE DOS SEUS DADOS

#DEFINA A VARIAVEL y para a qual será construído o histograma. 
#Basta substituir o valor entre aspas pelo nome da coluna onde está Y:

y = dataset['Y']

#NOTA: o valor y necessariamente deve ser uma série.

df = testes_normalidade(y = y, alpha = 0.10)

"""
Substitua o valor alpha de acordo com as instruções dadas na CÉLULA 11. 

"""

df

"""VOCÊ DESEJA EXPORTAR OS DADOS?
Caso deseje exportar os dados, copie a seguinte linha para o espaço não-vermelho após as aspas. Substitua o endereço pela
pasta onde você deseja salvar seu arquivo. Substitua dataframe pelo nome que deseja para seu arquivo. Mantenha
a extensão csv

df.to_csv(r"D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 3 - ANN classification\ANN.Class - 1 - user satisfaction\dataframe.csv", index = False)

"""

In [ ]:
## CÉLULA 13: Função para realizar a transformação Box-Cox dos dados (transformar os dados originais em dados que sejam 
representados por uma curva normal)

#Execute esta CÉLULA para carregar a função

def transf_box_cox(x, y, nome_x = None, limites_de_especificacao = None):
    import pandas as pd
    from statsmodels.stats import diagnostic
    from scipy import stats
    
    lambda_boxcox = stats.boxcox_normmax(y, method='pearsonr')
    #calcula o lambda da transformacao box-cox utilizando o metodo da maxima verossimilhanca
    #por meio da maximizacao do coeficiente de correlacao de pearson da funcao
    #y = boxcox(x), onde boxcox representa a transformacao
    
    #lista apenas com o lambda
    lista_lambda = []
    lista_lambda.append(lambda_boxcox)
    #APENAS LISTAS PODEM ENTRAR CORRETAMENTE NO DICIONARIO
    #Passo necessario para criacao dos novos dataframes
    
    #Calculo da variavel transformada
    y_transform = stats.boxcox(y, lmbda=lambda_boxcox, alpha=None)
    #Calculo da transformada
    
    if not (nome_x is None):
        #apenas se existe o nome da variavel X
        xlabel = nome_x
    
    else:
        xlabel = "X"
        
    d1 = {xlabel: x, 'Dados transformados': y_transform}
    #dicionario dos dados transformados
    df1 = pd.DataFrame(data = d1)
    #dataframe contendo os dados transformados
    
    #testes de normalidade da variavel transformada
    #Lilliefors’ test
    lilliefors_test = diagnostic.kstest_normal(y, dist='norm', pvalmethod='table')
    #Return: linha 1: ksstat: float
    #Kolmogorov-Smirnov test statistic with estimated mean and variance.
    #Linha 2: p-value:float
    #If the pvalue is lower than some threshold, e.g. 0.10, then we can reject the Null hypothesis that the sample comes from a normal distribution.
    
    #criar lista apenas com o p-valor
    p_lillie = []
    p_lillie.append(lilliefors_test[1])
    #apenas o p-valor na lista
    
    #Anderson-Darling
    ad_test = diagnostic.normal_ad(y, axis=0)
    #Return: Linha 1: ad2: float
    #Anderson Darling test statistic.
    #Linha 2: p-val: float
    #The p-value for hypothesis that the data comes from a normal distribution with unknown mean and variance.
    
    #criar lista apenas com o p-valor
    p_ad = []
    p_ad.append(ad_test[1])
    #apenas o p-valor na lista
    
    d2 = {'Lambda da transformação Box-Cox': lista_lambda, 'p-valor teste Lilliefors': p_lillie, 'p-valor teste Anderson-Darling': p_ad}
    #dicionario dos p-valores e do lambda
    #Apenas possivel ao se criar as listas de valores individuais
    df2 = pd.DataFrame(data = d2)
    #dataframe dos p-valores e do lambda, cada um em uma coluna de nome apropriado
    
    if not (limites_de_especificacao is None):
        #apenas executa este passo quando o limite de especificação for fornecido
        
        novos_limites = []
        i = 0
        while (i <2): 
            #a lista possui apenas os indices 0 e 1, correspondentes ao limite
            #inferior e ao limite superior
            if (limites_de_especificacao[i] == 0):
                novos_limites.append(0)
                #evitar o erro do exponencial igual a zero
            else:
                novos_limites.append(((limites_de_especificacao[i])**lambda_boxcox-1)/lambda_boxcox)
                #aplica a transformada aos limites
            
            #a lista novos_limites grava os limites de especificação transformados
            i = i + 1
        
        #aqui temos mais uma vez o problema da criação do dicionario
        #poderiamos simplesmente retornar a lista, mas não seria possível separar
        #colunas com nomes. Para isso, fazemos:
            
        lista_inf = []
        lista_inf.append(novos_limites[0])
        #apenas o limite inferior
        
        lista_sup = []
        lista_sup.append(novos_limites[1])
        # assim, garantimos que se um dos limites for zero, nao havera erro
        #na funcao exponencial, pois checamos tanto o inferior quanto o superior
        # e podemos criar um dataframe com as colunas indicadas por nomes
        
        d3 = {'Limite de especificação inferior transformado': lista_inf,
              'Limite de especificação superior transformado': lista_sup}
        
        df3 = pd.DataFrame(data = d3)
        #dataframe dos novos limites de especificação
    
    if not (limites_de_especificacao is None):
        #Caso haja limites de especificacao, retorna os limites transformados
        return df1, df2, df3
    
    #caso nao haja limite de especificacao:
    else:
        return df1, df2

# CÉLULA 14: Função para transformação Box-Cox

### ATENÇÃO: NÃO EXECUTE ESTA CÉLULA - Ela contém apenas texto explicativo
    
    
Utilize este sistema para transformar dados que não seguem a normal em dados que são representados por uma distribuição normal.

INSTRUÇÕES:

1) Esta função retorna 2 ou 3 dataframes:

        1 dataframe df1 contendo os dados transformados (SEMPRE)
        1 dataframe df2 contendo os p-valores dos testes de normalidade dos dados transformados (SEMPRE)
        1 dataframe df3 contendo os limites de especificação transformados (apenas quando forem fornecidos limites de especificação).

2) Caso não sejam fornecidos limites de especificação, chame dois dataframes

Exemplo: caso deseje salvar o dataframe df1 em y1, o dataframe df2 em y2 e os dados a serem transformados estão na variável y:
    
        y1, y2 = transf_box_cox(x, y, nome_x = None, limites_de_especificacao = None)
    
Você pode dar outros nomes para os dataframes chamados:
    
        dados_transformados, p_vals = transf_box_cox(x, y, nome_x = None, limites_de_especificacao = None)
        
3) Os parâmetros x e y da função devem ser apenas listas, não dataframes.
        
        Forneça os valores originais do eixo X
    
4) Caso haja limites de especificação, declare-os como uma lista de 2 valores

Exemplos: se a especificação é de 10 a 20 kg
    
        limites_de_especificacao = [10, 20]

- se a especificação está entre 0 a 12.5 L:

        limites_de_especificacao = [0, 12.5]

-Se não houver limites de especificação, manter:
    
        limites_de_especificacao = None

-A função retornará os limites já transformados

5) CASO SEJAM FORNECIDOS LIMITES DE ESPECIFICAÇÃO, você precisará chamar 3 dataframes:

        y1, y2, y3 = transf_box_cox(x, y, nome_x = None, limites_de_especificacao = [lim_inf, lim_sup])

Assim, supondo que os limites sejam 10 e 20 kg, e a variável seja y

        y1, y2, y3 = transf_box_cox(x, y, nome_x = None, limites_de_especificacao = [10, 20])

ou com os nomes desejados, por exemplo:
    
        dados_transformados, p_vals, lim_transformados = transf_box_cox(x, y, nome_x = None, limites_de_especificacao = [10, 20])

6) Caso não queira fornecer o nome da variável X, mantenha nome_x = None
    Caso queira, coloque o nome entre aspas. 
    Por exemplo: nome_x = "Dias da semana" ou
    nome_x = "Coleta"


In [ ]:
## CÉLULA 15: TRANSFORMAÇÃO BOX-COX DOS SEUS DADOS

#DEFINA A VARIAVEL X. Basta substituir o valor entre aspas pelo nome da coluna onde está X:
x = dataset['X']

#DEFINA A VARIAVEL y. Basta substituir o valor entre aspas pelo nome da coluna onde está Y:
y = dataset['Y']

"""
Se os seus dados possuírem limites de especificação a serem definidos, substitua a linha do comando imediatamente após
as aspas pela seguinte linha. Isso porque a linha do comando resulta em 2 dataframes, mas será necessária uma terceira
saída para os novos limites de especificação.


dados_transformados, p_vals, lim_transformados = transf_box_cox(x, y, nome_x = None, limites_de_especificacao = [lim_inf, lim_sup])



- Não se esqueça de substituir lim_inf pelo valor do limite inferior de especificação, e lim_sup pelo do limite superior.
Esta substituição não é necessária se as variáveis lim_inf e lim_sup forem definidas e tiverem seus valores especificados antes
de chamar a função.
Lembre-se que os limites_de_especificacao devem ser fornecidos no formato [2.71, 3.50] 
(dois valores sequenciais, separados por vírgula, e entre colchetes. O primeiro é o limite de especificação inferior, 
e o segundo é o superior).

"""

dados_transformados, p_vals = transf_box_cox(x = x, y = y, nome_x = None, limites_de_especificacao = None)


"""
Substitua os demais campos None de acordo com as instruções dadas na CÉLULA 14. 
- O nome da variável X deve ser FORNECIDO ENTRE ASPAS. Por exemplo: nome_x = "x"

"""

dados_transformados
p_vals

#acrescente mais uma linha contendo apenas lim_transformados caso estes valores tenham sido calculados. Basta remover o #
#da próxima linha (ela deixará de ser um comentário e se tornará um comando):

#lim_transformados

"""VOCÊ DESEJA EXPORTAR OS DADOS?
Caso deseje exportar os dados, copie as seguintes linhas para o espaço não-vermelho após as aspas. Substitua o endereço pela
pasta onde você deseja salvar seu arquivo. Substitua data_transform, p_valores, e lim_transform pelos nomes que deseja para seus arquivos. 
Mantenha a extensão csv. Você pode também optar por exportar apenas uma das tabelas (neste caso, copie apenas a desejada).

dados_transformados.to_csv(r"D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 3 - ANN classification\ANN.Class - 1 - user satisfaction\data_transform.csv", index = False)

p_vals.to_csv(r"D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 3 - ANN classification\ANN.Class - 1 - user satisfaction\p_valores.csv", index = False)

lim_transformados.to_csv(r"D:\Drive\FM2S\EAD\Green Belt em Python\Machine Learning - módulo 3 - ANN classification\ANN.Class - 1 - user satisfaction\lim_transform.csv", index = False)

"""